<a href="https://colab.research.google.com/github/louispranata/UTS_ML2019_12741423/blob/master/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 2: Practical Machine Learning Project

#Introduction

This report will be addressing a real-world problem and tries to solve it by producing analysis of its data and building data models. Based on the data, they will be tested accordingly and be compared with each other to provide concrete evidences of the solutions to the problems. For this report, it will be created by the cloud computing environment of Google Colab and using by the Python programming language.

The dataset that will be used to solve the problem is the results.csv which contains over 40000 international football results from the year 1872 to now. Each rows contain some corresponding attributes such as date, the teams, score line, tournament and venue. This file will be used as the input  of the data which is used to help to solve the problem.  

In [0]:
import numpy as np 
import pandas as pd
import xarray as xr  
import numpy as np

In [0]:
import csv

with open('results.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')

In [0]:
df = pd.read_csv('results.csv')
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [0]:
df.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
40834,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Apia,Samoa,True
40835,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Apia,Samoa,True
40836,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Cairo,Egypt,True
40837,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India,True
40838,2019-07-20,Papua New Guinea,Fiji,1,1,Pacific Games,Apia,Samoa,True


In [0]:
len(df)

40839

In [0]:
df['country'].unique()

array(['Scotland', 'England', 'Wales', 'Republic of Ireland',
       'United States', 'Uruguay', 'Austria', 'Hungary', 'Argentina',
       'Belgium', 'France', 'Netherlands', 'Bohemia', 'Switzerland',
       'Sweden', 'Germany', 'Italy', 'Chile', 'Norway', 'Finland',
       'Luxembourg', 'Russia', 'Denmark', 'Spain', 'Japan', 'Brazil',
       'Paraguay', 'China PR', 'Canada', 'Estonia', 'Guatemala',
       'Czechoslovakia', 'Poland', 'Yugoslavia', 'New Zealand', 'Romania',
       'Latvia', 'Portugal', 'Northern Ireland', 'Australia', 'Lithuania',
       'Turkey', 'Mexico', 'Aruba', 'Soviet Union', 'Haiti',
       'Philippines', 'Bulgaria', 'Jamaica', 'Kenya', 'Czech Republic',
       'Peru', 'Honduras', 'British Guyana', 'Uganda', 'El Salvador',
       'Barbados', 'Irish Free State', 'Trinidad and Tobago', 'Greece',
       'Curaçao', 'Egypt', 'Dominica', 'Guadeloupe', 'Palestine',
       'Netherlands Guyana', 'French Guiana', 'Cuba', 'Colombia',
       'Saint Kitts and Nevis', 'Éire', 

In [0]:
df['country'].value_counts()

United States               1144
France                       801
England                      687
Malaysia                     644
Sweden                       637
Germany                      581
Brazil                       529
Spain                        517
Thailand                     483
Italy                        480
Switzerland                  477
Austria                      475
United Arab Emirates         472
South Africa                 470
Qatar                        467
South Korea                  453
Argentina                    449
Hungary                      431
Chile                        405
Belgium                      396
Mexico                       392
Egypt                        391
Netherlands                  390
Norway                       385
Scotland                     385
Uruguay                      374
Poland                       372
Denmark                      371
Trinidad and Tobago          369
Singapore                    368
          

In [0]:
x = df['country'].unique()
print(x)


['Scotland' 'England' 'Wales' 'Republic of Ireland' 'United States'
 'Uruguay' 'Austria' 'Hungary' 'Argentina' 'Belgium' 'France'
 'Netherlands' 'Bohemia' 'Switzerland' 'Sweden' 'Germany' 'Italy' 'Chile'
 'Norway' 'Finland' 'Luxembourg' 'Russia' 'Denmark' 'Spain' 'Japan'
 'Brazil' 'Paraguay' 'China PR' 'Canada' 'Estonia' 'Guatemala'
 'Czechoslovakia' 'Poland' 'Yugoslavia' 'New Zealand' 'Romania' 'Latvia'
 'Portugal' 'Northern Ireland' 'Australia' 'Lithuania' 'Turkey' 'Mexico'
 'Aruba' 'Soviet Union' 'Haiti' 'Philippines' 'Bulgaria' 'Jamaica' 'Kenya'
 'Czech Republic' 'Peru' 'Honduras' 'British Guyana' 'Uganda'
 'El Salvador' 'Barbados' 'Irish Free State' 'Trinidad and Tobago'
 'Greece' 'Curaçao' 'Egypt' 'Dominica' 'Guadeloupe' 'Palestine'
 'Netherlands Guyana' 'French Guiana' 'Cuba' 'Colombia'
 'Saint Kitts and Nevis' 'Éire' 'Panama' 'Bohemia and Moravia' 'Slovakia'
 'Manchuria' 'Croatia' 'Costa Rica' 'Afghanistan' 'Martinique'
 'Southern Rhodesia' 'Iceland' 'Albania' 'Madagascar' 'Nor

In [0]:
england_data = df.loc[df['country'] == "England"]
england_data.head(5)